In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import plot_tree

from sklearn.model_selection import train_test_split
# Regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
# Clasification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, accuracy_score
# Best_searches
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# Standarization
from sklearn.preprocessing import StandardScaler

from keras import models
from keras.layers import Dense

In [2]:
df_train = pd.read_csv('data/train.csv', encoding='utf8')
df_test = pd.read_csv('data/test.csv', encoding='utf8')
df_final = pd.read_csv('data/sample_submission.csv', encoding='utf8')

In [3]:
df_train.describe()

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
mean,68214.000000,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,136428.000000,304.400000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
df_test[df_test['PWF'] != 0]

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
405,136834,L47380,L,298.4,308.6,2636,12.8,103,0,0,1,0,0
576,137005,L56355,L,297.9,309.0,1403,64.2,0,0,0,1,0,0
950,137379,L49935,L,299.7,309.3,1478,38.8,8,0,0,1,0,0
1570,137999,L55609,L,299.0,310.4,2861,4.6,92,0,0,1,0,0
2653,139082,L53074,L,301.2,311.0,1267,69.3,164,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86883,223312,M20301,M,303.0,312.6,1363,62.9,119,0,0,1,0,0
87045,223474,L49512,L,299.2,308.5,2232,15.9,220,0,0,1,0,0
89247,225676,M18633,M,302.4,311.1,2659,10.8,139,0,0,1,0,0
90239,226668,L51961,L,303.4,311.7,1374,63.7,121,0,0,1,0,0


In [5]:
df_train.shape

(136429, 14)

In [6]:
df_train[df_train['Machine failure'] != 0].shape

(2148, 14)

## Opis danych: 

id - 
    Wiemy o co chodzi
    
Podcut ID - 
    Pomysły: Zmapować konkretny produkt id, jako taki z dużą lub małą szansą na uszkodzenie. 
    
Type - ( H, M, L )(High, Medium, Low) - 
    Im wyższy tym mniejsza szansa na uszkodzenie i większa tolerancja na długotrwałą pracę
    
Air temparture - temperatura powietrza
    zakres: 295 - 304 stopnie, median 300
    Różnica między tą temperaturą a temperaturą procesu może wskazywać na przegrzewanie.
    Pomysły: Zakodować różnicę między temperaturami.
    
Process temperature - temperatura pracy
    zakres 305 - 314, median 310.
    Pomysły: Sprawdzić zależność między awariami a tą temperaturą, DLA WSZYSTKICH TYPU AWARII
    
<i>Jeżeli Air Temprature będzie równe Process temp lub większe, użądzenie nie będzie się w stanie schłodzić i się nagrzeje.</i> 

Rotational speed rpm - prędkość rotacji/obrotu
    zakres 1181 - 2886, 996 unikalnych
    Pomysł: Porównać z Tool Wear 
    
Torque Nm - Moment obrotowy 
    Pomysł: użyć wzór, spróbować oszacować błąd. Power (kW) = Torque (N.m) x Speed (RPM) z TWF
    Możliwe: powiązanie OSF
    
Tool wear - Czas sesji użycia narzędzia
    Zakres 0 - 253
    Pomysł: Im większy tool wear, może powodować awarie
    
Typy awarii:
    TWF - Zużycie
    HDF - Przegrzanie
    PWF - Za mała moc
    OSF - Przeciążenie mechaniczne
    RNF - Losowe
    Pomysły: Zobaczyć zależności pomiędzy tymi typami awarii i dokładnie je zbadać.

In [7]:
df_train.describe()

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
mean,68214.000000,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,136428.000000,304.400000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
Power (kW) = Torque (N.m) x Speed (RPM)

## Get to know your data

In [23]:
df_train['Product ID'].unique().shape #(9976,)
df_train['Type'].unique() #array(['L', 'M', 'H'], dtype=object)
df_train['Air temperature [K]'] #(95,)
df_train['Process temperature [K]'].unique().shape #(81,)
df_train['Tool wear [min]'].unique().shape #(246,)
df_train['Rotational speed [rpm]'].unique().shape #(952,)
df_train['Torque [Nm]'].unique().shape #(611,)
df_train['Tool wear [min]'].unique().shape #(246,)

(246,)